# Project 4: Dynamic Ticket Pricing – Regression & What-Ifs
Predict **tickets_sold** and explore pricing scenarios to maximize revenue.

### Tasks
1. Train **GradientBoostingRegressor** (or RandomForest) to predict demand.
2. Evaluate with MAE/RMSE; inspect feature importances.
3. Build a simple price optimization loop to find price that maximizes `price * predicted_demand` for an upcoming game.

In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

df = pd.read_csv('../datasets/ticket_sales_dynamic_pricing.csv')
X = df[['opponent_popularity_1_10','weekday','days_to_game','discount_pct','price']]
y = df['tickets_sold']
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_tr, y_tr)
pred = gbr.predict(X_te)
print({'MAE': mean_absolute_error(y_te, pred), 'RMSE': mean_squared_error(y_te, pred, squared=False), 'R2': r2_score(y_te, pred)})

# Feature importance
fi = pd.Series(gbr.feature_importances_, index=X.columns).sort_values(ascending=False)
fi.plot(kind='bar', title='Feature Importance')
plt.show()


In [ ]:
# Simple price optimization for a hypothetical game
game = pd.DataFrame({
    'opponent_popularity_1_10':[8],
    'weekday':[6],  # Sunday
    'days_to_game':[14],
    'discount_pct':[10],
    'price':[400]
})
prices = np.arange(200, 900, 10)
rev = []
for p in prices:
    g = game.copy(); g['price'] = p
    demand = gbr.predict(g)[0]
    rev.append(p * max(0, demand))
best_idx = int(np.argmax(rev))
print({'best_price': float(prices[best_idx]), 'max_revenue': float(rev[best_idx])})
import matplotlib.pyplot as plt
import pandas as pd
pd.DataFrame({'price':prices, 'revenue':rev}).plot(x='price', y='revenue', title='Price vs Revenue')
plt.show()
